In [1]:
import numpy as np
import pickle
import keras
from keras import models, optimizers, layers, regularizers, metrics
from keras.models import model_from_json
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)
import matplotlib.pyplot as plt

IMG_SIZE = 256

Using TensorFlow backend.


In [2]:
# Save baseline model architecture and weights
def save_base(model_name):
    model_str = str(input("Save model as: "))

    # serialize model to JSON
    model_name = baseline.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    baseline.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Save Comparison model
def save_model(model_name):
    model_str = str(input("Save model as: "))

    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model

# Write history object to a file using pickle
def save_history(model_name):
    hist_str = str(input("Save history as: "))

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

In [3]:
training_img_1 = pickle.load(open('training_img_1.p', 'rb'))
training_img_2 = pickle.load(open('training_img_2.p', 'rb'))
training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('pickle_test_img.p', 'rb'))

training_labels = pickle.load(open('training_labels.p', 'rb'))
test_labels = pickle.load(open('test_labels.p', 'rb'))

# Vanilla ConvNets

### VGG_Doppleganger

In [ ]:
# Directly taken from VGG16 with 2D Spatial Dropout, Dropout, and fewer Dense layers
'''
accuracy
'''
VGG_Doppleganger = models.Sequential()

# Block 1
VGG_Doppleganger.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
VGG_Doppleganger.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))

# Block 2
VGG_Doppleganger.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
VGG_Doppleganger.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))

# Block 4
VGG_Doppleganger.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))

# Block 5
VGG_Doppleganger.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.Dropout(0.5))

# Output Block
VGG_Doppleganger.add(layers.Flatten())
VGG_Doppleganger.add(layers.Dense(64, activation='relu'))
VGG_Doppleganger.add(layers.Dense(15, activation='sigmoid'))

VGG_Doppleganger.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

VGG_Doppleganger_history = VGG_Doppleganger.fit(training_img, training_labels, epochs = 40, validation_split = (1 / 9), batch_size = 128, verbose = 1)

# VGG_Doppleganger

save_history(VGG_Doppleganger_history)
save_model(VGG_Doppleganger_history)

In [4]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
'''
accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

VGG_Doppleganger_v2_history = model.fit(training_img, training_labels, epochs = 40, validation_split = (1 / 9), batch_size = 128, verbose = 1)

# VGG_Doppleganger_v2

Train on 89696 samples, validate on 11212 samples
Epoch 1/40
89696/89696 [==============================] - 267s 3ms/step - loss: 0.6713 - acc: 0.9203 - val_loss: 0.4376 - val_acc: 0.9223
Epoch 2/40
89696/89696 [==============================] - 263s 3ms/step - loss: 0.3224 - acc: 0.9246 - val_loss: 0.3226 - val_acc: 0.9213
Epoch 3/40
89696/89696 [==============================] - 266s 3ms/step - loss: 0.2732 - acc: 0.9252 - val_loss: 0.2860 - val_acc: 0.9227
Epoch 4/40
89696/89696 [==============================] - 271s 3ms/step - loss: 0.2515 - acc: 0.9257 - val_loss: 0.2686 - val_acc: 0.9194
Epoch 5/40
89696/89696 [==============================] - 270s 3ms/step - loss: 0.2392 - acc: 0.9260 - val_loss: 0.2549 - val_acc: 0.9201
Epoch 6/40
89696/89696 [==============================] - 274s 3ms/step - loss: 0.2311 - acc: 0.9262 - val_loss: 0.2400 - val_acc: 0.9208
Epoch 7/40
89696/89696 [==============================] - 270s 3ms/step - loss: 0.2258 - acc: 0.9265 - val_loss: 0.2361 - 

In [7]:
save_history(VGG_Doppleganger_v2_history)
save_model(VGG_Doppleganger_v2_history)

Save history as: VGG_Doppleganger_v2_history
Saved VGG_Doppleganger_v2_history to Training Histories folder
Save model as: VGG_Doppleganger_v2_40e
Saved VGG_Doppleganger_v2_40e and weights to CNN Models folder


In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
# v3: Using Categorical Accuracy as metrics
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

VGG_Doppleganger_v3_history = model.fit(training_img, training_labels, epochs = 40, validation_split = (1 / 9), batch_size = 128)

# VGG_Doppleganger_v3

### vanilla_VGG_Dropouts

In [5]:
# Based on VGG with half reg'd layers and Dropout layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_VGG_Dropouts_history = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

'''
vanilla_VGG_Dropouts
history: vanilla_VGG_Dropouts_history
model: vanilla_VGG_Dropouts_30e
'''

Train on 89696 samples, validate on 11212 samples
Epoch 1/30
89696/89696 [==============================] - 226s 3ms/step - loss: 0.4351 - categorical_accuracy: 0.5198 - val_loss: 0.3634 - val_categorical_accuracy: 0.5100
Epoch 2/30
89696/89696 [==============================] - 222s 2ms/step - loss: 0.2909 - categorical_accuracy: 0.5335 - val_loss: 0.3845 - val_categorical_accuracy: 0.5022
Epoch 3/30
89696/89696 [==============================] - 221s 2ms/step - loss: 0.2550 - categorical_accuracy: 0.5327 - val_loss: 0.3738 - val_categorical_accuracy: 0.4959
Epoch 4/30
89696/89696 [==============================] - 222s 2ms/step - loss: 0.2407 - categorical_accuracy: 0.5314 - val_loss: 0.3251 - val_categorical_accuracy: 0.4973
Epoch 5/30
89696/89696 [==============================] - 222s 2ms/step - loss: 0.2322 - categorical_accuracy: 0.5307 - val_loss: 0.2977 - val_categorical_accuracy: 0.4913
Epoch 6/30
89696/89696 [==============================] - 221s 2ms/step - loss: 0.2265 - c

In [8]:
save_history(vanilla_VGG_Dropouts_history)
save_model(vanilla_VGG_Dropouts_history)

Save history as: vanilla_VGG_Dropouts_history
Saved vanilla_VGG_Dropouts_history to Training Histories folder
Save model as: vanilla_VGG_Dropouts_30e
Saved vanilla_VGG_Dropouts_30e and weights to CNN Models folder


In [6]:
# v2: Added 2D Sp Dropout to Block 1
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_VGG_Dropouts_v2_history = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

'''
 vanilla_VGG_Dropouts_v2
 history: vanilla_VGG_Dropouts_v2_history
 model: vanilla_VGG_Dropouts_v2_30e
'''

Train on 89696 samples, validate on 11212 samples
Epoch 1/30
89696/89696 [==============================] - 230s 3ms/step - loss: 0.4561 - categorical_accuracy: 0.5301 - val_loss: 0.5632 - val_categorical_accuracy: 0.5098
Epoch 2/30
89696/89696 [==============================] - 228s 3ms/step - loss: 0.3209 - categorical_accuracy: 0.5365 - val_loss: 0.4090 - val_categorical_accuracy: 0.5098
Epoch 3/30
89696/89696 [==============================] - 229s 3ms/step - loss: 0.2763 - categorical_accuracy: 0.5335 - val_loss: 0.3647 - val_categorical_accuracy: 0.5060
Epoch 4/30
89696/89696 [==============================] - 228s 3ms/step - loss: 0.2543 - categorical_accuracy: 0.5309 - val_loss: 0.3250 - val_categorical_accuracy: 0.5039
Epoch 5/30
89696/89696 [==============================] - 227s 3ms/step - loss: 0.2413 - categorical_accuracy: 0.5298 - val_loss: 0.2790 - val_categorical_accuracy: 0.5083
Epoch 6/30
89696/89696 [==============================] - 232s 3ms/step - loss: 0.2326 - c

In [9]:
save_history(vanilla_VGG_Dropouts_v2_history)
save_model(vanilla_VGG_Dropouts_v2_history)

Save history as: vanilla_VGG_Dropouts_v2_history
Saved vanilla_VGG_Dropouts_v2_history to Training Histories folder
Save model as: vanilla_VGG_Dropouts_v2_30e
Saved vanilla_VGG_Dropouts_v2_30e and weights to CNN Models folder


In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
vanilla_VGG_Dropouts_v3 = models.Sequential()

# Block 1
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v3.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v3.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v3.add(layers.Dropout(0.5))

# Output Block
vanilla_VGG_Dropouts_v3.add(layers.Flatten())
vanilla_VGG_Dropouts_v3.add(layers.Dense(64, activation='relu'))
vanilla_VGG_Dropouts_v3.add(layers.Dense(15, activation='sigmoid'))


vanilla_VGG_Dropouts_v3.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_VGG_Dropouts_v3_history = vanilla_VGG_Dropouts_v3.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_VGG_Dropouts_v3

save_history(vanilla_VGG_Dropouts_v3_history)
save_model(vanilla_VGG_Dropouts_v3_history)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
vanilla_VGG_Dropouts_v4 = models.Sequential()

# Block 1
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v4.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v4.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v4.add(layers.Dropout(0.5))

# Output Block
vanilla_VGG_Dropouts_v4.add(layers.Flatten())
vanilla_VGG_Dropouts_v4.add(layers.Dense(64, activation='relu'))
vanilla_VGG_Dropouts_v4.add(layers.Dense(15, activation='sigmoid'))


vanilla_VGG_Dropouts_v4.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_VGG_Dropouts_v4_history = vanilla_VGG_Dropouts_v4.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_VGG_Dropouts_v4

save_history(vanilla_VGG_Dropouts_v4_history)
save_model(vanilla_VGG_Dropouts_v4_history)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
vanilla_VGG_Dropouts_v5 = models.Sequential()

# Block 1
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v5.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v5.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v5.add(layers.Dropout(0.375))

# Block 4
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v5.add(layers.Dropout(0.375))

# Block 5
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v5.add(layers.Dropout(0.375))

# Output Block
vanilla_VGG_Dropouts_v5.add(layers.Flatten())
vanilla_VGG_Dropouts_v5.add(layers.Dense(64, activation='relu'))
vanilla_VGG_Dropouts_v5.add(layers.Dense(15, activation='sigmoid'))


vanilla_VGG_Dropouts_v5.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_VGG_Dropouts_v5_history = vanilla_VGG_Dropouts_v5.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_VGG_Dropouts_v5

save_history(vanilla_VGG_Dropouts_v5_history)
save_model(vanilla_VGG_Dropouts_v5_history)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
# v6: Added Batch Normalization layers to each Block
vanilla_VGG_Dropouts_v6 = models.Sequential()

# Block 1
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v6.add(layers.BatchNormalization())
vanilla_VGG_Dropouts_v6.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v6.add(layers.BatchNormalization())
vanilla_VGG_Dropouts_v6.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v6.add(layers.BatchNormalization())
vanilla_VGG_Dropouts_v6.add(layers.Dropout(0.375))

# Block 4
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v6.add(layers.BatchNormalization())
vanilla_VGG_Dropouts_v6.add(layers.Dropout(0.375))

# Block 5
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts_v6.add(layers.BatchNormalization())
vanilla_VGG_Dropouts_v6.add(layers.Dropout(0.375))

# Output Block
vanilla_VGG_Dropouts_v6.add(layers.Flatten())
vanilla_VGG_Dropouts_v6.add(layers.Dense(64, activation='relu'))
vanilla_VGG_Dropouts_v6.add(layers.Dense(15, activation='sigmoid'))


vanilla_VGG_Dropouts_v6.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_VGG_Dropouts_v6_history = vanilla_VGG_Dropouts_v6.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_VGG_Dropouts_v6

save_history(vanilla_VGG_Dropouts_v6_history)
save_model(vanilla_VGG_Dropouts_v6_history)

### vanilla_reg_VGG

In [ ]:
# Modeled after VGGNet with half reg'd layers
vanilla_reg_VGG = models.Sequential()

# Block 1
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 2
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 3
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Output Block
vanilla_reg_VGG.add(layers.Flatten())
vanilla_reg_VGG.add(layers.Dense(64, activation='relu'))
vanilla_reg_VGG.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_reg_VGG_history = vanilla_reg_VGG.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_reg_VGG

save_history(vanilla_reg_VGG_history)
save_model(vanilla_reg_VGG_history)

In [ ]:
# v2: Added reg's to all layers
'''
categorical_accuracy
'''
vanilla_reg_VGG_v2 = models.Sequential()

# Block 1
vanilla_reg_VGG_v2.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG_v2.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.MaxPooling2D((2, 2)))

# Block 2
vanilla_reg_VGG_v2.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.MaxPooling2D((2, 2)))

# Block 3
vanilla_reg_VGG_v2.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_reg_VGG_v2.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_reg_VGG_v2.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v2.add(layers.MaxPooling2D((2, 2)))

# Output Block
vanilla_reg_VGG_v2.add(layers.Flatten())
vanilla_reg_VGG_v2.add(layers.Dense(64, activation='relu'))
vanilla_reg_VGG_v2.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG_v2.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_reg_VGG_v2_history = vanilla_reg_VGG_v2.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_reg_VGG_v2

save_history(vanilla_reg_VGG_v2_history)
save_model(vanilla_reg_VGG_v2_history)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
vanilla_reg_VGG_v3 = models.Sequential()

# Block 1
vanilla_reg_VGG_v3.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG_v3.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.MaxPooling2D((2, 2)))

# Block 2
vanilla_reg_VGG_v3.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.MaxPooling2D((2, 2)))

# Block 3
vanilla_reg_VGG_v3.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_reg_VGG_v3.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_reg_VGG_v3.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v3.add(layers.MaxPooling2D((2, 2)))

# Output Block
vanilla_reg_VGG_v3.add(layers.Flatten())
vanilla_reg_VGG_v3.add(layers.Dense(32, activation='relu'))
vanilla_reg_VGG_v3.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG_v3.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_reg_VGG_v3_history = vanilla_reg_VGG_v3.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_reg_VGG_v3

save_history(vanilla_reg_VGG_v3_history)
save_model(vanilla_reg_VGG_v3_history)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
# v4: Added Batch Normalization to all layers
vanilla_reg_VGG_v4 = models.Sequential()

# Block 1
vanilla_reg_VGG_v4.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG_v4.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG_v4.add(layers.BatchNormalization())

# Block 2
vanilla_reg_VGG_v4.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG_v4.add(layers.BatchNormalization())

# Block 3
vanilla_reg_VGG_v4.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG_v4.add(layers.BatchNormalization())

# Block 4
vanilla_reg_VGG_v4.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG_v4.add(layers.BatchNormalization())

# Block 5
vanilla_reg_VGG_v4.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG_v4.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG_v4.add(layers.BatchNormalization())

# Output Block
vanilla_reg_VGG_v4.add(layers.Flatten())
vanilla_reg_VGG_v4.add(layers.Dense(32, activation='relu'))
vanilla_reg_VGG_v4.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG_v4.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

vanilla_reg_VGG_v4_history = vanilla_reg_VGG_v4.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_reg_VGG_v4

save_history(vanilla_reg_VGG_v4_history)
save_model(vanilla_reg_VGG_v4_history)